In [23]:
% run items.py
import pandas as pd 
import numpy as np

In [24]:
items_df = pd.DataFrame(items)
items_df

,border,color,form,letter
0,False,blue,square,E
1,True,red,circle,A
2,False,green,circle,B
3,True,red,diamond,E
4,True,red,diamond,E
5,False,blue,diamond,C
6,False,green,square,A
7,True,green,diamond,D
8,True,silver,star,C
9,False,silver,square,C


In [25]:
it_ar = np.array(items_df.drop('border', axis=1))
tar_ar = np.array(items_df['border'])

In [26]:
# probability is the same
# weights are the same
# need to get entropy the same
p, e, we = entropy_on_split(items_df, items_df['border'], 'color', debug=False)

In [27]:
for prop in p:
    print(entropy(prop))

1.0
-0.0
0.918295834054
0.918295834054
-0.0


In [38]:
zero = .1**10
myp[myp == 0] = zero

In [20]:
np.array(p[0])

array([ 0.5,  0.5])

In [44]:
entropy(myp[0])

1.0

In [50]:
my_entropy(myp[1])

0.91829583405448956

In [51]:
for p in myp:
    print(entropy(p))

1.0
0.918295834054
3.32192809489e-09
3.32192809489e-09
0.918295834054


In [34]:
entropy(p[0])

1.0

In [28]:
myw, myp = entropy_split(it_ar, tar_ar,0)

[array([False, False,  True,  True, False,  True], dtype=bool),
 array([False, False,  True, False, False,  True], dtype=bool),
 array([False], dtype=bool),
 array([ True,  True,  True,  True,  True], dtype=bool),
 array([ True, False, False], dtype=bool)]

[0.2857142857142857,
 0.2857142857142857,
 0.047619047619047616,
 0.23809523809523808,
 0.14285714285714285]

array([[ 0.5       ,  0.5       ],
       [ 0.33333333,  0.66666667],
       [ 0.        ,  1.        ],
       [ 1.        ,  0.        ],
       [ 0.33333333,  0.66666667]])

In [49]:
def my_entropy(proportions):
    
    log_2_proportions = np.log2(proportions)
    return -proportions.dot(log_2_proportions)



In [22]:
def entropy_split(data, target, feature, debug=True):
    
    unique_classes, class_counts = np.unique(data[:,feature], return_counts=True)
    
    target_subsets = [
        target[data[:,feature] == unique_class]
        for unique_class in unique_classes]

    if debug: display(target_subsets)
        
    total = target.shape[0]
    weights = [class_count / total for class_count in class_counts]
    
    if debug: display(weights)
        
    proportions = get_class_proportions(target_subsets)
    
    if debug: display(proportions)
        
#     entropies = my_entropy(proportions)
    
    return weights, proportions

    '''proportions = [
        get_class_proportions(target_subset)
        for target_subset in target_subsets
    ]
    
    if debug:
        display(proportions)
    '''

In [7]:
def class_proportions(target):
        
    total_count = target.count()
    class_count = target.value_counts()
    
    return class_count / total_count


def entropy(proportions):
    proportions = np.array(proportions)
    log_2_proportions = np.log2(proportions)
    return -proportions.dot(log_2_proportions)


def entropy_on_split(dataframe, target, feature, debug=False):
    
    # split on feature
    unique_classes = dataframe[feature].unique()
    
    target_subsets = [
        target[dataframe[feature] == unique_class]
        for unique_class in unique_classes]
    
    if debug:
        display(target_subsets)
        
    # calculate subset weights
    total = target.count()
    weights = [
        target_subset.count()/total
        for target_subset in target_subsets
    ]
    if debug:
        display(weights)
        
    # calculate target proprotions
    proportions = [
        class_proportions(target_subset)
        for target_subset in target_subsets
    ]
    if debug: display(proportions)
        
    # calculate the entropies
    entropies = [
        entropy(proportion)
        for proportion in proportions
    ]
    weighted_entropies = [
        weight*entropy
        for weight, entropy in zip(weights,entropies)
    ]
    return proportions, entropies, weighted_entropies

def classify(tree, element):
    if tree in [True, False]:
        return tree
    
    # if tuple defining a subtree dictionary 
    # attribute defines the split
    # subtree dictionary defines the subtrees created by that split
    attribute, subtree_dictionary = tree
    
    # get the attribute from the element associated with that split
    subtree_key = element.get(attribute)
    
    if subtree_key not in subtree_dictionary:
        subtree_key = None
    
    #get the subtree associated with the key 
    subtree = subtree_dictionary[subtree_key]
    
    return classify(subtree, element)


def find_best_split(features, target):
    feature_labels = features.columns
    entropies = [
        entropy_on_split(features, target, feature_label)
        for feature_label in feature_labels
    ]
    
    best_index = np.argmin(entropies)
    return feature_labels[best_index]


def build_tree(features, target, split_candidates=None):
    #handle the case for split_candidates is None
    if split_candidates is None:
        split_candidates = list(features.columns)
        
    total_count = target.count()
    true_count = target.sum()
    false_count = total_count - true_count
    
    if false_count == 0: return True
    if true_count == 0: return False
    
    if split_candidates == []:
        return true_count > false_count
    
    # find the best attribute to split upon
    best_attribute = find_best_split(features, target, split_candidates)
    
    # remove best attribute from list of split candidates
    split_candidates = [split_candidate
                       for split_candidate in split_candidates
                       if split_candidate is not best_attribute]
    
    best_attribute_uniques = features[best_attribute].unique()
    
    subtree_dictionary = dict()
    
    for best_attribute_unique in best_attribute_uniques:
        feat_subset_mask = features[best_attribute] == best_attribute_unique
        feat_subset = features[feat_subset_mask]
        target_subset = target[feat_subset_mask]
        
        subtree_dictionary[best_attribute_unique] = \
            build_tree(feat_subset, target_subset, split_candidates)
            
    subtree_dictionary[None] = true_count > false_count
        
    return(best_attribute, subtree_dictionary)
    
            

In [21]:
def get_class_proportions(target):
    
    prop = []
    
    for i in range(len(target)):
        prob_true = (np.sum(target[i] / len(target[i])))
        prob_false = 1 - prob_true
        prob = [prob_true, prob_false]
        
        prop.append(prob)
            
    return np.array(prop)


In [ ]:
fig = plt.figure(figsize=(10,8))
plt.scatter(X,y)
plt.xlabel('X', fontsize = 20)
plt.ylabel('y', fontsize = 20)
plt.title('Our Regression Model with Residuals!', fontsize=30)
plt.xlim((0,22))
plt.ylim((0,120))
plt.plot(X,y_pred, c='r', label='Our Linear Model!')
for xx, yy, yp in zip(X, y, y_pred):
    plt.plot([xx,xx],[yy,yp], c='g')
plt.show()

In [ ]:
def build_kdtree(points, depth = 0):
    n = len(points)
    
    if n = 0:
        raise ValueError("Data set must have a length greater than 0")
    
    axis = depth % k 
    sorted_points = sorted(points, key=lambda point: point[axis])
    
    return { 
    'point': sorted_points[n / 2 ]
    'left': build_kdtree(sorted_points[:n / 2], depth +1)}